# Task:

### You are required to convert VGG formatted JSON annotations file into COCO format. Additionally you are have to augment that images.

In [11]:
# Import required Libraries

import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import imutils
import math
import random

In [12]:
# Changing the directory
import os
os.chdir(r'C:\Users\uzair\Desktop\Computer Vision\cat_dog')
os.getcwd()

'C:\\Users\\uzair\\Desktop\\Computer Vision\\cat_dog'

In [13]:
# Opening the given VGG annotation json file
with open('cat_dog_annotations.json', 'r') as f:
    data = json.load(f)

In [15]:
# Creating a new directory for task
if not os.path.exists('CV_Evaulation_Task'):
    os.makedirs('CV_Evaluation_Task')

In [16]:
# Initializing the COCO Format
coco = {
    'info': {},
    'licenses': [],
    'images' : [],
    'annotations' : [],
    'categories': []
}

In [17]:
# Adding the information in info key
coco["info"] = {
    
    "year": 2023,
    "version": 1.0,
    "description": "Dataset in COCO Format",
    "contributor": "Uzair Naeem",
    "url": "ml1.ai",
    "date_created": "Mon Mar 27 2023 00:00:00 GMT+0500 (Pakistan Standard Time)"
}

coco['info']

{'year': 2023,
 'version': 1.0,
 'description': 'Dataset in COCO Format',
 'contributor': 'Uzair Naeem',
 'url': 'ml1.ai',
 'date_created': 'Mon Mar 27 2023 00:00:00 GMT+0500 (Pakistan Standard Time)'}

In [18]:
# Adding the information in license key
license = {
    'id' : 1,
    'name' : 'Unknown',
    'url' : ''
}

coco['licenses'].append(license)

coco['licenses']

[{'id': 1, 'name': 'Unknown', 'url': ''}]

In [19]:
# Performing augmentation 

image_id = 1
annotation_id = 0
categories = {}
category_id = 1
j = 1

for img_key, img_data in data['_via_img_metadata'].items():
    filename = img_data['filename']
    regions = img_data['regions']
    
    angle = 5
    # Opening the image
    img = cv2.imread(filename)
    # Getting the height and width of image
    height, width = img.shape[:2]
    image_id -= 1
    
    for i in range(72):
        # Defining scale factor between 60 to 100 percent
        scale_factor = np.random.uniform(0.6, 1)
        # Calculating the scaled height and width
        new_height = int(height * scale_factor)
        new_width = int(width * scale_factor)
        # Resizing the image according to scaled factor
        resized_img = cv2.resize(img, (new_width, new_height))
        # Calculating the height and width of resized image for bounding box 
        (img_height, img_width) = resized_img.shape[:2]
        (cx, cy) = (img_width//2, img_height//2)
        # Adding random brightness to image
        contrast = 1
        brightness = random.uniform(1.1, 1.5)
        bright_image = cv2.addWeighted(resized_img, brightness, resized_img, 0, contrast)
        # Performing Rotation of brightened image while preserving the edges
        rot_matrix = cv2.getRotationMatrix2D((cx,cy), i * angle, 1)
        cos = np.abs(rot_matrix[0, 0])
        sin = np.abs(rot_matrix[0, 1])
        nW = int((img_height * sin) + (img_width * cos))
        nH = int((img_height * cos) + (img_width * sin))
        rot_matrix[0, 2] += (nW / 2) - cx
        rot_matrix[1, 2] += (nH / 2) - cy
        rotated_image = cv2.warpAffine(bright_image, rot_matrix, (nW, nH))
        
        # Loop through the regions and draw bounding boxes
        for region in regions:
            shape_attributes = region['shape_attributes']
            region_attributes = region['region_attributes']
            label = list(region_attributes['class'].keys())[0]
            # Extracting x,y, width and height info from the image
            x, y = shape_attributes['x'], shape_attributes['y']
            w, h = shape_attributes['width'], shape_attributes['height']
            
            if label not in categories:
                categories[label] = category_id
                category_id += 1
                
            # Rescaling them according to resized image  
            scaled_x = int(x * scale_factor)
            scaled_y = int(y * scale_factor)
            scaled_w = int(w * scale_factor)
            scaled_h = int(h * scale_factor)
            
            # Calculating the array of four coordinates of bounidng box
            points = np.array([[scaled_x, scaled_y], [scaled_x + scaled_w, scaled_y], [scaled_x + scaled_w, scaled_y + scaled_h], [scaled_x, scaled_y + scaled_h]])
            # Rotating these points according to the rotation angle of image
            rotated_points = np.dot(rot_matrix, np.vstack([points.T, np.ones((1, 4))]))
            # Extracting top left and bottom right coordinates
            min_x = np.min(rotated_points[0])
            max_x = np.max(rotated_points[0])
            min_y = np.min(rotated_points[1])
            max_y = np.max(rotated_points[1])
            # Calculating new width and height of rotated and resized bounding box
            new_x = int(min_x)
            new_y = int(min_y)
            new_w = int(max_x - min_x)
            new_h = int(max_y - min_y)
            # Drawing rectangle for the bounding boxes
            cv2.rectangle(rotated_image, (new_x, new_y), (new_x + new_w, new_y + new_h), (0, 0, 255), 2)

            # Calculating all four coordinates of rotated bounding box for segmentation
            x1, y1 = new_x, new_y
            x2, y2 = new_x + new_w, y1
            x3, y3 = new_x + new_w, new_y + new_h
            x4, y4 = x1, new_y + new_h
            # Segmentation
            segmentation = [x1, y1, x2, y2, x3, y3, x4, y4]
            # Bounding Box
            bbox = [x1, y1, x3, y3]
            # Calculating Area
            area = new_w * new_h
            
            # Annotation key for cooc format
            annotation = {
                
                    'id' : annotation_id,
                    'image_id' : image_id,
                    'category_id' : categories[label],
                    'segmentation' : segmentation,
                    'area' : width * height,
                    'bbox' : bbox,
                    'iscrowd' : 0
                }
            
            coco['annotations'].append(annotation)
            
            annotation_id += 1
        
        # Saving the new augmented image in directory 
        img_name =  f"{j}_{i*angle}degrees.png"
        cv2.imwrite(f"CV_Evaluation_Task/{img_name}", rotated_image)
        
        # Image key for cooc format
        image = {
            'id' : image_id,
            'width' : nW,
            'height' : nH,
            'file_name' : img_name,
            'license' : 1,
            'date_captured': ''
        }
        
        coco['images'].append(image)
        image_id += 1
        
    image_id += 1    
    j += 1
    
    
for cls, id in categories.items():
    coco["categories"].append({
        "id": id,
        "name": cls,
        "supercategory": "class"
    })
    

with open(os.path.join("CV_Evaluation_Task","annotations.json"), "w") as f:
    json.dump(coco, f)

In [10]:
# Saved COCO format annotation 
with open(r'CV_Evaluation_Task\annotations.json') as f:
    my_data = json.load(f)
    
my_data

{'info': {'year': 2023,
  'version': 1.0,
  'description': 'Dataset in COCO Format',
  'contributor': 'Uzair Naeem',
  'url': 'ml1.ai',
  'date_created': 'Mon Mar 27 2023 00:00:00 GMT+0500 (Pakistan Standard Time)'},
 'licenses': [{'id': 1, 'name': 'Unknown', 'url': ''}],
 'images': [{'id': 0,
   'width': 716,
   'height': 537,
   'file_name': '1_0degrees.png',
   'license': 1,
   'date_captured': ''},
  {'id': 1,
   'width': 558,
   'height': 438,
   'file_name': '1_5degrees.png',
   'license': 1,
   'date_captured': ''},
  {'id': 2,
   'width': 496,
   'height': 406,
   'file_name': '1_10degrees.png',
   'license': 1,
   'date_captured': ''},
  {'id': 3,
   'width': 593,
   'height': 503,
   'file_name': '1_15degrees.png',
   'license': 1,
   'date_captured': ''},
  {'id': 4,
   'width': 740,
   'height': 647,
   'file_name': '1_20degrees.png',
   'license': 1,
   'date_captured': ''},
  {'id': 5,
   'width': 699,
   'height': 630,
   'file_name': '1_25degrees.png',
   'license': 1,
